## Lets work on creating a programmatic way to receive clip full videos into clips of individual swings

### To generate clipped dataset, we want to utilize:
    1) csv with the appropriate information about original videos, swings
    2) ffmpeg commands that will dictate how the video is decoded/encoded to generate the clips
         - Since videos are from an iphone --> there are some peculiarities to consider: 
             - https://www.perplexity.ai/search/8176d69e-475a-4e7f-ba07-0b762adb7c65
    We'll then create an appropriate clipped folder within folder holding all the full videos from a specific date

In [1]:
#| include: false
from fastai.vision.all import *
from IPython.display import Video
import subprocess
import ffmpeg
import cv2

In [45]:
base_path = '../../../data/full_videos'
swing_days = ['jun8', 'aug9', 'sep14']
files = get_files(f'{base_path}/{swing_days[0]}', extensions='.MOV')
len(files), files[0]

(13, Path('../../../data/full_videos/jun8/IMG_0851.MOV'))

In [137]:
df_jun8 = pd.read_csv(f'{base_path}/{swing_days[0]}/cleaned.csv').reset_index(drop=True)
df_aug9 = pd.read_csv(f'{base_path}/{swing_days[1]}/cleaned.csv').reset_index(drop=True)
df_sep14 = pd.read_csv(f'{base_path}/{swing_days[2]}/cleaned.csv').reset_index(drop=True)
df_jun8.head(1)

,input_file,swing_index,score,start,end,output_file
0,IMG_0848.MOV,0,2,00:20,00:23,IMG_0848_swing_0_score_2.mp4


In [174]:
# just checking that the dataframe columns are the same
assert((df_aug9.columns == df_jun8.columns).sum() == len(df_aug9.columns))
assert((df_aug9.columns == df_sep14.columns).sum() == len(df_aug9.columns))

In [272]:
## All the necessary code to take a dataframe and clip the videos however we need

def clip_videos(df, 
                input_path, 
                crf='18',
                video_encoder='copy',
                debug=False):
    output_folder_path = f'{input_path}/crf_{crf}_vcodec_{video_encoder}'
    for idx in range(len(df)):
        make_clip(input_folder_path=input_path, 
                  output_folder_path=output_folder_path,
                  row=df.iloc[idx],
                 crf=crf,
                 vcodec=video_encoder)
        if debug: break
    assert(len(os.listdir(output_folder_path)) == len(df))

def make_output_filename(row):
    return f'{row.input_file.split(".")[0]}_swing_{row.swing_index}_score_{row.score}.mp4'

def make_clip(input_folder_path, 
              output_folder_path,
              row, 
              time='0:03',
              crf='18',
              vcodec='copy'):
    input_file_name = row.input_file
    output_file_name = make_output_filename(row)
    start = row.start
    input_file_path = f'{input_folder_path}/{input_file_name}'
    output_file_path = f'{output_folder_path}/{output_file_name}'
    if os.path.isdir(output_folder_path) is False:
        os.mkdir(output_folder_path)
    ffmpeg.input(input_file_path, 
                 ss=start)\
        .output(output_file_path, 
                t=time, 
                vcodec=vcodec,
                crf=crf, 
                acodec='aac',
                y=None) \
        .global_args('-movflags', '+faststart') \
        .run()

In [269]:
clip_videos(df_jun8, input_path=f'{base_path}/{swing_days[0]}')

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [270]:
clip_videos(df_aug9, input_path=f'{base_path}/{swing_days[1]}')

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [271]:
clip_videos(df_sep14, input_path=f'{base_path}/{swing_days[2]}')

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

## Now some code to pull down the frames

In [82]:
def get_frames(swing_path, 
               resize=True, 
               width=256, 
               height=256,
               debug=False):
    capture = cv2.VideoCapture(swing_path)
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_array = np.empty((frame_count, frame_height, frame_width, 3), dtype=np.uint8)
    idx = 0
    while idx < frame_count:
        ret, frame = capture.read()
        if not ret:
            break
        video_array[idx] = frame
        idx += 1

    capture.release()
    if debug:
        print(video_array.shape)
    video_array = [convert_rgb(frame) for frame in video_array]
    if resize:
        video_array = np.array([resize_frame(frame, width, height) for frame in video_array])
    return video_array

def resize_frame(frame, width=256, height=256):
    return cv2.resize(frame, (width, height))

def convert_rgb(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

def plot_three(image1, image2, image3=None):
    # Create figure with 1 row, 3 columns of subplots
    if image3 is None:
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    else:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # Plot each image on its respective axis
    axes[0].imshow(image1)
    axes[0].set_title('Frame 1')
    axes[0].axis('off')  # Remove axis ticks and labels

    axes[1].imshow(image2)
    axes[1].set_title('Frame 2')
    axes[1].axis('off')

    if image3 is not None:
        axes[2].imshow(image3)
        axes[2].set_title('Frame 3 or Diff')
        axes[2].axis('off')

    plt.tight_layout()  # Adjust spacing between subplots
    plt.show()

In [5]:
Video('testing.mp4')

In [23]:
Video('data/clips/testing/IMG_0848_swing_0_score_2.mp4')